## Tensorized Composition Algorithm

In [1]:
from ten_con import mat_decompose_xz
from pauli_utils import *

In [2]:
from copy import deepcopy
import numpy as np
from numba import jit

In [3]:
def fro_inner(A:np.matrix, B:np.matrix):
    r"""Frobineous inner product of two given square matrices, :code:`A` and :code:`B`.

    .. math:
        \langle A | B \rangle = \frac{1}{n} A^\dagger B
    Args:
        A (np.matrix): Square numpy matrix.
        B (np.matrix): Square numpy matrix.

    Returns:
        _type_: _description_
    """
    
    na1, na2 = A.shape
    nb1, nb2 = B.shape
    assert na1 == na2, "First matrix was not a square matrix."
    assert nb1 == nb2, "Second matrix was not a square matrix."
    assert na1 == nb1, f"The dimension was not matched, {na1}, {nb1}."
    return np.trace(A.H @ B)/na1

In [4]:
def mat_decompose(H:np.matrix):
    #Tensrosized reconstruction method: O(8^n)
    # Normal method: O(16^n)
    # mat = np.zeros(self.coef_matrix.shape) 
    # for p in self.poly:
    #   mat += p.coef*p.matrix
    #mat = self.coef_matrix
    n1, n2 = H.shape
    assert n1 == n2, "The given matrix must be a square matrix."
    n= int(np.log2(n1))
    l = n1
    for i in range(n):
        m = int(2**i) # Number of submatrix
        l = int(l/2) # Sub matrix size, square
        for j in range(m):
            for k in range(m):
                num_i = j*(2*l) # Initial position of sub matrix row
                num_j = k*(2*l) # Initial position of sub matrix column
                
                # There is no problem of inplace operators.
                
                # I-Z
                H[num_i: num_i+l, num_j:num_j+l]        += H[num_i+l: num_i+2*l, num_j+l:num_j+2*l] 
                H[num_i+l: num_i+2*l, num_j+l:num_j+2*l] = H[num_i: num_i+l, num_j:num_j+l] - 2*H[num_i+l: num_i+2*l, num_j+l:num_j+2*l]
                # X-Y
                H[num_i: num_i+l, num_j+l:num_j+2*l] +=  H[num_i+l: num_i+2*l, num_j:num_j+l] 
                H[num_i+l: num_i+2*l, num_j:num_j+l]  =  H[num_i: num_i+l, num_j+l:num_j+2*l] - 2*H[num_i+l: num_i+2*l, num_j:num_j+l]
                H[num_i+l: num_i+2*l, num_j:num_j+l] *= 1j

    H *= (1/(2**n))
    return H
def mat_compose(mat:np.matrix):
    _2n = mat.shape[0] # 2^n
    steps = int(np.log2(_2n))# n
    unit_size= 1
 
    for step in range(steps):
        step1 = step+1
        mat_size = int(2*(unit_size))
        indexes = np.arange(_2n/(2**step1)).astype(np.uint)
        indexes_ij = (mat_size * indexes)
        for i in indexes_ij:
            for j in indexes_ij:
                # (i, j)
                r1i     = int(i)
                r1f2i   = int(r1i + unit_size)
                c1i     = int(j)
                c1f2i   = int(c1i + unit_size)
                r2f     = int(r1f2i + unit_size)
                c2f     = int(c1f2i + unit_size)

                #print(i, j, unit_size+1, "|",  r1i, r1f2i, c1i, c1f2i, r2f, c2f)

                # Do not replace the below code to in-place operator += or *=.
                # Numba jit yieds different handling process in compile time. 
                # I - Z
                coef = 1
                mat[r1i: r1f2i, c1i:c1f2i] = mat[r1i: r1f2i, c1i:c1f2i] + coef*mat[r1f2i: r2f, c1f2i:c2f]
                mat[r1f2i: r2f, c1f2i:c2f] = mat[r1i: r1f2i, c1i:c1f2i] -2*coef *mat[r1f2i: r2f, c1f2i:c2f]
                # X -Y
                coef = -1j
                mat[r1i: r1f2i, c1f2i:c2f] = mat[r1i: r1f2i, c1f2i:c2f]  + coef*mat[r1f2i: r2f, c1i:c1f2i]
                mat[r1f2i: r2f, c1i:c1f2i] = mat[r1i: r1f2i, c1f2i:c2f] - 2*coef*mat[r1f2i: r2f, c1i:c1f2i]
                
        unit_size *=2
    return mat

##  Random Hamiltonian test

In [22]:
from scipy.sparse import coo_matrix
from copy import deepcopy
from qiskit.quantum_info import SparsePauliOp

In [33]:
# Random Hamiltonian generation
qubits = 7
N = int(2**qubits)
A = np.random.rand(N, N)
B = np.random.rand(N, N)
C = np.matrix(A + 1j*B)
H = (C.H@C)

coef_mat = (mat_decompose_xz(deepcopy(H)))
coef_mat_pre = mat_decompose(deepcopy(H))
coef_mat_pre1 = deepcopy(coef_mat_pre)
#tolerance = 0
#coef_mat.real[np.abs(coef_mat.real) < tolerance] = 0
#coef_mat.imag[np.abs(coef_mat.imag) < tolerance] = 0

h_mat = coo_matrix(coef_mat)
pauli_list = np.stack([h_mat.row, h_mat.col], axis=1)
pstrs = []
for p, c in zip(pauli_list, h_mat.data):
    pstr = sym_code2pstr(p, qubits)
    pstrs.append((pstr, c))

In [34]:

print("H=", end="")
for i, p in enumerate(pstrs):
    print(f"c{i:03}", p[0], "+", end="")
    if i %50 ==0:
        print()
print("...")

print("Pauli term:", len(pauli_list))

H=c000 IIIIIII +
c001 IIIIIIZ +c002 IIIIIZI +c003 IIIIIZZ +c004 IIIIZII +c005 IIIIZIZ +c006 IIIIZZI +c007 IIIIZZZ +c008 IIIZIII +c009 IIIZIIZ +c010 IIIZIZI +c011 IIIZIZZ +c012 IIIZZII +c013 IIIZZIZ +c014 IIIZZZI +c015 IIIZZZZ +c016 IIZIIII +c017 IIZIIIZ +c018 IIZIIZI +c019 IIZIIZZ +c020 IIZIZII +c021 IIZIZIZ +c022 IIZIZZI +c023 IIZIZZZ +c024 IIZZIII +c025 IIZZIIZ +c026 IIZZIZI +c027 IIZZIZZ +c028 IIZZZII +c029 IIZZZIZ +c030 IIZZZZI +c031 IIZZZZZ +c032 IZIIIII +c033 IZIIIIZ +c034 IZIIIZI +c035 IZIIIZZ +c036 IZIIZII +c037 IZIIZIZ +c038 IZIIZZI +c039 IZIIZZZ +c040 IZIZIII +c041 IZIZIIZ +c042 IZIZIZI +c043 IZIZIZZ +c044 IZIZZII +c045 IZIZZIZ +c046 IZIZZZI +c047 IZIZZZZ +c048 IZZIIII +c049 IZZIIIZ +c050 IZZIIZI +
c051 IZZIIZZ +c052 IZZIZII +c053 IZZIZIZ +c054 IZZIZZI +c055 IZZIZZZ +c056 IZZZIII +c057 IZZZIIZ +c058 IZZZIZI +c059 IZZZIZZ +c060 IZZZZII +c061 IZZZZIZ +c062 IZZZZZI +c063 IZZZZZZ +c064 ZIIIIII +c065 ZIIIIIZ +c066 ZIIIIZI +c067 ZIIIIZZ +c068 ZIIIZII +c069 ZIIIZIZ +c070 ZIIIZZI +c0

In [35]:
qiskit_restore = (SparsePauliOp.from_list(pstrs)).to_matrix()
tpc_restore = mat_compose(coef_mat_pre1)
D1 = np.matrix((H - qiskit_restore))
D2 = np.matrix((H - tpc_restore))
print("Difference(Qiskit):", fro_inner(D1, D1))
print("Difference(TPC):", fro_inner(D2, D2))

Difference(Qiskit): (1.0736065415935064e-25+0j)
Difference(TPC): (2.0450202088881157e-26+0j)


In [36]:
%%timeit
qiskit_restore = (SparsePauliOp.from_list(pstrs)).to_matrix()

782 ms ± 9.99 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%%timeit
tpc_restore = mat_compose(deepcopy(coef_mat_pre))

60.3 ms ± 417 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
